BU PROJEDE DATA İÇERİSİNDE BULUNAN TEXT VE LABEL SÜTUNLARINI KULLANARAK YAZILAN YAZIYA GÖRE KİŞİNİN STRESLİ OLUP OLMADIĞINA İLİŞKİN BİR SINIFLANDIRMA MODELİ EĞİTMEYE ÇALIŞACAĞIZ

In [60]:
import pandas as pd
import nltk
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [61]:
data = pd.read_csv('stress.csv')

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Columns: 116 entries, subreddit to sentiment
dtypes: float64(106), int64(6), object(4)
memory usage: 2.5+ MB


In [63]:
data.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [64]:
#SADECE TEXT VE LABEL SÜTUNLARINI ALIYORUZ.
data2 = data[['text','label']]

In [65]:
from nltk.stem import WordNetLemmatizer

In [66]:
#KÖKLERİ VE STOPWORDLERİ BULMAK İÇİN AŞAĞIDAKİ KODLARI YAZIYORUZ.
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

In [67]:
#NUMARALARI AŞAĞIDAKİ GİBİ ETİKETLENDİRİYORUZ.
data2["label"] = data2["label"].map({0: "No Stress", 1: "Stress"})

C:\Users\PC\AppData\Local\Temp\ipykernel_21996\163238859.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2["label"] = data2["label"].map({0: "No Stress", 1: "Stress"})


In [68]:
#TEXT SÜTUNUNDA BULUNAN TEXTLER TEMİZLENİYOR.
def clean_text(text):
    text = str(text).lower() # harfleri küçültüyor.
    text = re.sub('/[.*?/]','',text) # mesela "[örnek]" ifadesini köşeli parantezlerde dahil olmak üzere tamamen siliyor. 
    text = re.sub('https?://\S+|www\.\S+','',text) # 'http veya https veya www ile başlayan tüm internet adreslerini siliyor.'
    text = re.sub('<.*?>+','',text) # html ifadeleri siliyor '<örnek>'
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #tüm noktalama işaretlerini siliyor. % ifadesi sağdaki noktalama işaretlerini soldaki köşeli parantezin içine almaya yarıyor.
    text = re.sub('\n', '', text) #'/n' ifadesi varsa siliyor kelime içerisinde.
    text = re.sub('\w*\d\w*', '', text) #içerisinde rakam olan kelimeleri yakalayıp siliyor.
    text = [word for word in text.split(' ') if word not in stopword] # text'i parçalıyor ve stop words varsa siliyor.
    text=" ".join(text) #stopwords silindikten sonra tekrar texti birleştiriyor.
    text = [lemmatizer.lemmatize(word) for word in text.split(' ')] #text'i tekrar ayırıyıyor ve köklerini bularak liste yapıyor.
    text=" ".join(text) #kökler bulunduktan sonra listeyi tekrar string hale getirip birleştiriyor.
    return text # texti döndürüyor.

data["text"] = data["text"].apply(clean_text)

In [69]:
x = np.array(data2['text'])
y = np.array(data2['label'])

In [70]:
vect = TfidfVectorizer()
x_vect = vect.fit_transform(x)

In [71]:
x_train, x_test, y_train, y_test = train_test_split(x_vect, y,test_size=0.10,random_state=25)

In [72]:
#İLK MODELİMİZİ EĞİTİYORUZ
Bb = BernoulliNB()
model = Bb.fit(x_train,y_train)

In [73]:
tahmin = Bb.predict(x_test)

In [74]:
accuracy_score(y_test, tahmin)

0.7323943661971831

In [75]:
#İKİNCİ MODELİMİZİ EĞİTİYORUZ.
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [76]:
tahmin2 = lr.predict(x_test)

In [77]:
accuracy_score(y_test, tahmin2)

0.7711267605633803

BU PROJEDE İKİ MODEL EĞİTTİK BernoulliNB İÇİN BAŞARI ORANI %73 İKEN LogisticRegression İÇİN BAŞARI ORANI %77 CİVARINDA OLDU.